In [1]:
import pandas as pd

data = pd.read_csv('data.csv')

In [23]:
data = data[['category_name', 'title']]
sample = data.sample(frac=0.01)
data.head()

,category_name,title
0,Телефоны,Xiaomi RedMI 4a
1,Автомобили,"Audi Q5, 2009"
2,Детская одежда и обувь,Толстовка очень теплая
3,Телефоны,"Продаю айфон 6 16 гб сильвер с зарядкой,без ко..."
4,Аквариум,Аквариум


In [252]:
from sklearn.cluster import (
    AffinityPropagation, AgglomerativeClustering,
    DBSCAN, MeanShift, Birch
)
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn import metrics

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Задание 1

In [29]:
cv = CountVectorizer(max_features=500)
tfidf = TfidfVectorizer(max_features=500)

cv_vec = cv.fit_transform(sample['title'])
tfidf_vec = tfidf.fit_transform(sample['title'])

In [30]:
y = sample['category_name']

In [52]:
def get_affinity(vectors, **preferences):
    cluster = AffinityPropagation(**preferences)
    cluster.fit(vectors)
    labels = cluster.labels_
    return labels, vectors

In [57]:
def get_agglomerative(vectors, **preferences):
    svd = TruncatedSVD(50)
    vectors_svd = svd.fit_transform(vectors)
    cluster = AgglomerativeClustering(**preferences)
    cluster.fit(vectors_svd)
    labels = cluster.labels_
    return labels, vectors

In [58]:
def get_dbscan(vectors, **preferences):
    svd = TruncatedSVD(50)
    vectors_svd = svd.fit_transform(vectors)
    cluster = DBSCAN(**preferences)
    cluster.fit(vectors_svd)
    labels = cluster.labels_
    return labels, vectors

In [55]:
def print_metrics(labels, vectors):
    print("Silhouette Coefficient: %0.3f" 
          % metrics.silhouette_score(vectors, labels))
    print("Homogeneity: %0.3f" %
          metrics.homogeneity_score(y, labels))
    print("Completeness: %0.3f"
          % metrics.completeness_score(y, labels))
    print("V-measure: %0.3f"
          % metrics.v_measure_score(y, labels))
    print("Adjusted Rand Index: %0.3f"
          % metrics.adjusted_rand_score(y, labels))
    print("Adjusted Mutual Information: %0.3f"
          % metrics.adjusted_mutual_info_score(y, labels))

In [119]:
labels, vectors = get_affinity(cv_vec, damping=0.7, preference=-5, max_iter=400, verbose=2)
#Да, параметры как в семинарской тетрадке, но так он даёт наиболее внятные результаты
print_metrics(labels, vectors)

Converged after 249 iterations.
Silhouette Coefficient: 0.197
Homogeneity: 0.362
Completeness: 0.383
V-measure: 0.372
Adjusted Rand Index: -0.026
Adjusted Mutual Information: 0.220


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [122]:
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 15].head()

,title,label_test,label
21126,Комбинезон-трансформер демисезонный,Детская одежда и обувь,15
84323,Комбинезон,Детская одежда и обувь,15
16726,Комбинезон kerry,Детская одежда и обувь,15
35710,Комбинезон,Детская одежда и обувь,15
59303,Комбинезон-трансформер,Детская одежда и обувь,15


In [143]:
labels, vectors = get_affinity(tfidf_vec, damping=0.8, preference=-7, max_iter=400, verbose=2)
print_metrics(labels, vectors)

Converged after 19 iterations.
Silhouette Coefficient: -0.336
Homogeneity: 0.530
Completeness: 0.299
V-measure: 0.383
Adjusted Rand Index: 0.016
Adjusted Mutual Information: 0.044


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [144]:
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 5].head()

,title,label_test,label
29650,Плуг нтз 82,Грузовики и спецтехника,5
24999,Грузчики,Предложение услуг,5
137421,Кожаная куртка -двойка,"Одежда, обувь, аксессуары",5
124858,Дом 120 м² на участке 15 сот.,"Дома, дачи, коттеджи",5
44916,Автомат по продаже кукол,Оборудование для бизнеса,5


### Вывод по `AffinityPropagation`
С `CountVectorizer` результаты гораздо более внятные. По метрикам не очень понятно. Они больше зависят от параметров обучения, чем от того, какой там тип векторов.

In [154]:
labels, vectors = get_agglomerative(cv_vec, n_clusters=50)
# Кластеров лучше брать меньше
# Метрики будут чуть ниже, зато результат красивее
# Если кластеров много, он просто начинает выискивать по словам
print_metrics(labels, vectors)

Silhouette Coefficient: 0.187
Homogeneity: 0.322
Completeness: 0.374
V-measure: 0.346
Adjusted Rand Index: -0.025
Adjusted Mutual Information: 0.205


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [157]:
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 0].head()

,title,label_test,label
41388,Платье для невесты,"Одежда, обувь, аксессуары",0
109338,Куртка для девочек,Детская одежда и обувь,0
14702,Ботиночки для девочки,Детская одежда и обувь,0
60032,Платье для принцессы,Детская одежда и обувь,0
128938,Ролики для девочки р. 27-30,Спорт и отдых,0


In [158]:
labels, vectors = get_agglomerative(tfidf_vec, n_clusters=50)
print_metrics(labels, vectors)

Silhouette Coefficient: 0.222
Homogeneity: 0.328
Completeness: 0.371
V-measure: 0.348
Adjusted Rand Index: -0.025
Adjusted Mutual Information: 0.208


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
50790,Ноутбук Sony Vaio,Ноутбуки,0
131412,Куплю дачу,"Дома, дачи, коттеджи",0
23986,"Отмостки, дорожки, площадки",Предложение услуг,0
72830,Нарды,Спорт и отдых,0
89581,Маз-543203-2122,Грузовики и спецтехника,0


In [160]:
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 1].head()

,title,label_test,label
21007,Демисезонный комплект для мальчика,Детская одежда и обувь,1
87866,Костюм для мальчика классический104 см,Детская одежда и обувь,1
54986,Свитер зимний,"Одежда, обувь, аксессуары",1
202,Комплект мама+дочка,Детская одежда и обувь,1
24278,Брюки на мальчика,Детская одежда и обувь,1


### Вывод по `AgglomerativeClustering`
Оба вырианта векторизации работают хорошо. У _tfidf_ чуть лучше метрики, так что формальную победу можно отдать ему.

In [168]:
labels, vectors = get_dbscan(cv_vec, min_samples=10, eps=0.3)
print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 1].head()

Silhouette Coefficient: 0.040
Homogeneity: 0.121
Completeness: 0.261
V-measure: 0.165
Adjusted Rand Index: 0.006
Adjusted Mutual Information: 0.081


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
108508,Продам духи,Красота и здоровье,1
2589,Продам пиджак,"Одежда, обувь, аксессуары",1
127833,Продам вазу,Мебель и интерьер,1
56370,Продам защитное стекло xiaomi redmi 3 3s 3x 3 pro,Телефоны,1
101142,Продам сланцы Крокс,"Одежда, обувь, аксессуары",1


In [170]:
labels, vectors = get_dbscan(tfidf_vec, min_samples=10, eps=0.3)
print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 1].head()

Silhouette Coefficient: 0.099
Homogeneity: 0.135
Completeness: 0.302
V-measure: 0.186
Adjusted Rand Index: -0.005
Adjusted Mutual Information: 0.087


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
108508,Продам духи,Красота и здоровье,1
127833,Продам вазу,Мебель и интерьер,1
4446,Продам барана,Другие животные,1
138706,Продам электрогенератор,Ремонт и строительство,1
34457,Продам хипсит,Товары для детей и игрушки,1


### Вывод по `DBSCAN`
Обы ищут по словам, так себе. Формальная победа за _tfidf_

# Задание 2

In [172]:
svd = TruncatedSVD(50)
vectors_svd = svd.fit_transform(vectors)

nmf = NMF(50)
vectors_nmf = nmf.fit_transform(vectors)

### `AgglomerativeClustering`

In [181]:
cluster = AgglomerativeClustering(n_clusters=100)
cluster.fit(vectors_svd)
labels = cluster.labels_

In [182]:
print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 5].head()

Silhouette Coefficient: 0.234
Homogeneity: 0.393
Completeness: 0.372
V-measure: 0.382
Adjusted Rand Index: -0.028
Adjusted Mutual Information: 0.202


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
96759,Ортопедические сандали,Детская одежда и обувь,5
35528,Сандали,Детская одежда и обувь,5
47105,"Сандали Milton, р. 21, цвет хаки",Детская одежда и обувь,5
88286,"Пляжная обувь,сандали резиновые",Детская одежда и обувь,5
63463,Сандали 23 размер,Детская одежда и обувь,5


In [183]:
cluster = AgglomerativeClustering(n_clusters=100)
cluster.fit(vectors_nmf)
labels = cluster.labels_

In [185]:
print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 5].head()

Silhouette Coefficient: 0.224
Homogeneity: 0.385
Completeness: 0.370
V-measure: 0.377
Adjusted Rand Index: -0.029
Adjusted Mutual Information: 0.198


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
48492,Платье 42 р-р,"Одежда, обувь, аксессуары",5
126702,Вечернее платье,"Одежда, обувь, аксессуары",5
136467,Платье детское,Детская одежда и обувь,5
40384,Летнее платье,"Одежда, обувь, аксессуары",5
13135,Вечернее платье,"Одежда, обувь, аксессуары",5


### `DBSCAN`

In [187]:
cluster = DBSCAN(min_samples=10, eps=0.3)
cluster.fit(vectors_svd)
labels = cluster.labels_

In [188]:
print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 5].head()

Silhouette Coefficient: 0.098
Homogeneity: 0.133
Completeness: 0.298
V-measure: 0.184
Adjusted Rand Index: -0.005
Adjusted Mutual Information: 0.086


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
114620,Куртка демисезонная,"Одежда, обувь, аксессуары",5
55225,Куртка сноубордическая exxtasy (42-44),"Одежда, обувь, аксессуары",5
60865,Куртка джинсовая Benetton,Детская одежда и обувь,5
31622,Куртка бомбер,"Одежда, обувь, аксессуары",5
27808,Куртка zara,"Одежда, обувь, аксессуары",5


In [189]:
cluster = DBSCAN(min_samples=10, eps=0.3)
cluster.fit(vectors_nmf)
labels = cluster.labels_

In [191]:
print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == 1].head()

Silhouette Coefficient: 0.085
Homogeneity: 0.023
Completeness: 0.256
V-measure: 0.042
Adjusted Rand Index: -0.012
Adjusted Mutual Information: 0.010


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
112493,Белые босоножки,"Одежда, обувь, аксессуары",1
127816,Босоножки,Детская одежда и обувь,1
86503,Анатомические Босоножки/ сандалии Тотто,Детская одежда и обувь,1
91071,Босоножки,Детская одежда и обувь,1
135509,Босоножки,Детская одежда и обувь,1


## Вывод

С `AgglomerativeClustering` разницы почти нет, с `DBSCAN` лучше работает `TruncatedSVD`

# Задание 3

In [214]:
labels, vectors = get_dbscan(tfidf_vec, min_samples=5, eps=0.4)
print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == -1].head(15)

Silhouette Coefficient: 0.158
Homogeneity: 0.161
Completeness: 0.317
V-measure: 0.213
Adjusted Rand Index: -0.050
Adjusted Mutual Information: 0.084


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
31867,Костюм демисезонный новый,Детская одежда и обувь,-1
21007,Демисезонный комплект для мальчика,Детская одежда и обувь,-1
132963,Новые сапожки,"Одежда, обувь, аксессуары",-1
45898,Сапоги весна-осень,Детская одежда и обувь,-1
10392,Конверт весна-осень,Детская одежда и обувь,-1
41388,Платье для невесты,"Одежда, обувь, аксессуары",-1
71880,Чехол,Телефоны,-1
17791,Фильтр новый на 100-200л,Аквариум,-1
99807,Ободки ручной работы из камней,Часы и украшения,-1
109338,Куртка для девочек,Детская одежда и обувь,-1


`DBSCAN` упорно кладёт одежду в выбросы, надо попробовать что-то ещё

In [239]:
svd = TruncatedSVD(50)
vectors_svd = svd.fit_transform(cv_vec)
cluster = MeanShift(cluster_all=False, bandwidth=0.1)
cluster.fit(vectors_svd)
labels = cluster.labels_

print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == -1].head(15)

Silhouette Coefficient: 0.260
Homogeneity: 0.542
Completeness: 0.365
V-measure: 0.436
Adjusted Rand Index: -0.036
Adjusted Mutual Information: 0.060


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label
79543,"Сервиз обеденный ""Domestic""",Посуда и товары для кухни,-1
56370,Продам защитное стекло xiaomi redmi 3 3s 3x 3 pro,Телефоны,-1
113663,Телевизор Sanyo,Аудио и видео,-1
8472,"Тунис, вылет 11/05 (все включено)",Билеты и путешествия,-1
37532,Ветровка,"Одежда, обувь, аксессуары",-1
40342,Столик пеленальный подвесной,Товары для детей и игрушки,-1
47198,Детский автомобиль в стиле ретро,Товары для детей и игрушки,-1
4152,Наушники Puma,Телефоны,-1
41065,Услуги грузчиков,Предложение услуг,-1
89014,Кольцо лягушки,Товары для детей и игрушки,-1


Опять никой дичи

In [253]:
cluster = Birch(n_clusters=40)
cluster.fit(vectors_svd)
labels = cluster.labels_

print_metrics(labels, vectors)
df = pd.DataFrame({
    'title': sample['title'],
    'label_test': sample['category_name'],
    'label': labels,
})
df[df['label'] == -1].head(15)

Silhouette Coefficient: 0.140
Homogeneity: 0.246
Completeness: 0.373
V-measure: 0.296
Adjusted Rand Index: -0.009
Adjusted Mutual Information: 0.158


/home/misha/.local/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


,title,label_test,label


Остальные тоже дичи почему-то не выдают :(